##### Command used to run docker image:

```bash
> pwd
/home/rav/projects/gwas-analysis
> sudo docker run --user $(id -u):$(id -g) --rm -ti -v $(pwd):/home/$USER/repos/gwas-analysis -v 
$HOME/.ivy2:/home/$USER/.ivy2 -p 8888:8888 -p 4040:4040 -p 4041:4041 -p 8787:8787 -p 8080:8080 -p 8079:8079 -p 8849:8849 -v $HOME/projects/:/home/$USER/data gwas-analysis
```

In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import os
import sys
import json
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from vm_utils import get_vm_info
json.loads(get_vm_info())

{'platform': 'Linux',
 'platform-release': '5.0.0-1034-gcp',
 'platform-version': '#35-Ubuntu SMP Tue Mar 17 03:56:45 UTC 2020',
 'architecture': 'x86_64',
 'processor': 8,
 'processor_mhz': '2000.174',
 'ram': '31 GB'}

In [2]:
import hail as hl
import pandas as pd
import numpy as np
import plotnine as pn
import matplotlib.pyplot as plt
hl.init() 

Running on Apache Spark version 2.4.4
SparkUI available at http://2175f48b66f7:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.32-a5876a0a2853
LOGGING: writing to /home/rav/repos/gwas-analysis/notebooks/benchmark/method/pcrelate/hail-20200424-1805-0.2.32-a5876a0a2853.log


In [3]:
from pathlib import Path
import shutil
import glob
import os

def write_mt(path: Path, n_samples: int, n_variants: int, n_populations: int) -> None:
    mt = hl.balding_nichols_model(n_populations=n_populations, n_samples=n_samples, n_variants=n_variants)
    mt.write(path.as_posix(), overwrite=True)
    
def get_mt(n_samples: int, n_variants: int, n_populations: int) -> hl.MatrixTable:
    path_mt = Path(f"{Path.home().as_posix()}/data/tmp/mt_{n_samples}_{n_variants}_{n_populations}.mt")
    if not path_mt.exists():
        write_mt(path_mt, n_samples, n_variants, n_populations)
    return hl.read_matrix_table(path_mt.as_posix())

def clean_hail_tmp():
    dir_list = glob.iglob(Path("/tmp").joinpath("hail*").as_posix())
    for path in dir_list:
        if os.path.isdir(path):
            print(f"Deleting hail tmp dir: {path}")
            shutil.rmtree(path)

In [4]:
import time
import tempfile
from pathlib import Path
import shutil
from bench_utils import Experiment, Measurment

def do_experiment(experiment_name:str, samples: int, variants: int, populations: int, block_size: int = 4096) -> None:
    with Experiment(experiment_name) as experiment:
        try:
            mt = get_mt(n_samples=samples, n_variants=variants, n_populations=populations)
            # here we assume that there is no recent relatedness in the sample, which should not
            # matter for the purpose of the benchmark
            start = time.time()
            _, scores, _ = hl.methods.hwe_normalized_pca(mt.GT, k=10, compute_loadings=False)
            pca_time = time.time() - start
            print(f"PCA took s:{samples}, v:{variants}, p:{populations} : {pca_time:.2f} seconds")
            start = time.time()
            pc_relate_result = hl.methods.pc_relate(mt.GT, scores_expr=scores[mt.col_key].scores, min_individual_maf=0.0, block_size=block_size)
            tmp_dir = Path(tempfile.mkdtemp()).joinpath("pc_relate_result")
            pc_relate_result.write(tmp_dir.as_posix())
            pc_relate_time = time.time() - start
            print(f"Deleting pc_relate results: {tmp_dir.as_posix()}")
            n_rows = None
            shutil.rmtree(tmp_dir)
            experiment.add_measurment(Measurment(samples=samples, variants=variants, population=populations, pca_time=pca_time, pc_relate_time=pc_relate_time, num_rows=n_rows))
        finally:
            clean_hail_tmp()

In [9]:
do_experiment("test", 100, 100, 2)

2020-04-24 17:18:04 Hail: INFO: hwe_normalized_pca: running PCA using 100 variants.
2020-04-24 17:18:04 Hail: INFO: pca: running PCA with 10 components...


PCA took s:100, v:100, p:2 : 8.95 seconds


2020-04-24 17:18:11 Hail: INFO: Wrote all 1 blocks of 100 x 100 matrix with block size 4096.
2020-04-24 17:18:12 Hail: INFO: wrote matrix with 11 rows and 100 columns as 1 block of size 4096 to file:/tmp/hail.GXgR4Pd2q9HV/yJzccApuSF.bm
2020-04-24 17:18:12 Hail: INFO: wrote matrix with 100 rows and 100 columns as 1 block of size 4096 to file:/tmp/hail.GXgR4Pd2q9HV/0vxvtcMZCu.bm
2020-04-24 17:18:12 Hail: INFO: wrote matrix with 100 rows and 100 columns as 1 block of size 4096 to file:/tmp/hail.GXgR4Pd2q9HV/Mynco0KksJ.bm
2020-04-24 17:18:12 Hail: INFO: wrote matrix with 100 rows and 100 columns as 1 block of size 4096 to file:/tmp/hail.GXgR4Pd2q9HV/fGHDWn6jFU.bm
2020-04-24 17:18:12 Hail: INFO: wrote matrix with 100 rows and 100 columns as 1 block of size 4096 to file:/tmp/hail.GXgR4Pd2q9HV/CS5ZMks1ZU.bm
2020-04-24 17:18:12 Hail: INFO: wrote matrix with 100 rows and 100 columns as 1 block of size 4096 to file:/tmp/hail.GXgR4Pd2q9HV/eb8v54UndA.bm
2020-04-24 17:18:12 Hail: INFO: wrote matrix

Deleting pc_relate results: /tmp/tmpdgk8pxjw/pc_relate_result
Deleting hail tmp dir: /tmp/hail.lSI8hItnvyNe
Deleting hail tmp dir: /tmp/hail.GXgR4Pd2q9HV


In [ ]:
for s in range(1000, 10001, 1000):
    for v in range(1000, 10001, 1000):
        do_experiment("simple_pop_10_1000_to_10000_cross_cpu_8_ram_31", samples=s, variants=v, populations=10)

2020-04-23 15:27:42 Hail: INFO: balding_nichols_model: generating genotypes for 10 populations, 1000 samples, and 1000 variants...
2020-04-23 15:27:42 Hail: INFO: Coerced sorted dataset
2020-04-23 15:27:43 Hail: INFO: wrote matrix table with 1000 rows and 1000 columns in 8 partitions to /home/rav/data/tmp/mt_1000_1000_10.mt
2020-04-23 15:27:43 Hail: INFO: hwe_normalized_pca: running PCA using 1000 variants.
2020-04-23 15:27:44 Hail: INFO: pca: running PCA with 10 components...


PCA took s:1000, v:1000, p:10 : 6.28 seconds


2020-04-23 15:27:50 Hail: INFO: Wrote all 1 blocks of 1000 x 1000 matrix with block size 4096.
2020-04-23 15:27:51 Hail: INFO: wrote matrix with 11 rows and 1000 columns as 1 block of size 4096 to file:/tmp/hail.Y2Dh5hqGGSs3/cC5uLznRV3.bm
2020-04-23 15:27:51 Hail: INFO: wrote matrix with 1000 rows and 1000 columns as 1 block of size 4096 to file:/tmp/hail.Y2Dh5hqGGSs3/Y9jtcTTOB0.bm
2020-04-23 15:27:51 Hail: INFO: wrote matrix with 1000 rows and 1000 columns as 1 block of size 4096 to file:/tmp/hail.Y2Dh5hqGGSs3/0PrhSCPADZ.bm
2020-04-23 15:27:51 Hail: INFO: wrote matrix with 1000 rows and 1000 columns as 1 block of size 4096 to file:/tmp/hail.Y2Dh5hqGGSs3/uXck1hlyN0.bm
2020-04-23 15:27:51 Hail: INFO: wrote matrix with 1000 rows and 1000 columns as 1 block of size 4096 to file:/tmp/hail.Y2Dh5hqGGSs3/aiSKLu8693.bm
2020-04-23 15:27:52 Hail: INFO: wrote matrix with 1000 rows and 1000 columns as 1 block of size 4096 to file:/tmp/hail.Y2Dh5hqGGSs3/MM4z8nkuU7.bm
2020-04-23 15:27:52 Hail: INFO:

Deleting pc_relate results: /tmp/tmpcve4rxkd/pc_relate_result
Deleting hail tmp dir: /tmp/hail.Y2Dh5hqGGSs3


2020-04-23 15:28:04 Hail: INFO: wrote matrix table with 2000 rows and 1000 columns in 8 partitions to /home/rav/data/tmp/mt_1000_2000_10.mt
2020-04-23 15:28:04 Hail: INFO: hwe_normalized_pca: running PCA using 2000 variants.
2020-04-23 15:28:05 Hail: INFO: pca: running PCA with 10 components...


...

2020-04-23 18:05:03 Hail: INFO: Wrote all 4 blocks of 5000 x 6000 matrix with block size 4096.
2020-04-23 18:05:04 Hail: INFO: wrote matrix with 11 rows and 5000 columns as 2 blocks of size 4096 to file:/tmp/hail.Y2Dh5hqGGSs3/LSd3BLPnJv.bm
2020-04-23 18:05:05 Hail: INFO: wrote matrix with 5000 rows and 6000 columns as 4 blocks of size 4096 to file:/tmp/hail.Y2Dh5hqGGSs3/7XEoZscp1M.bm
2020-04-23 18:05:06 Hail: INFO: wrote matrix with 5000 rows and 6000 columns as 4 blocks of size 4096 to file:/tmp/hail.Y2Dh5hqGGSs3/pZG8obYzC1.bm
2020-04-23 18:05:23 Hail: INFO: wrote matrix with 6000 rows and 6000 columns as 4 blocks of size 4096 to file:/tmp/hail.Y2Dh5hqGGSs3/Kdw209uDrW.bm
2020-04-23 18:05:40 Hail: INFO: wrote matrix with 6000 rows and 6000 columns as 4 blocks of size 4096 to file:/tmp/hail.Y2Dh5hqGGSs3/9bauANUyAC.bm
2020-04-23 18:05:41 Hail: INFO: wrote matrix with 6000 rows and 6000 columns as 4 blocks of size 4096 to file:/tmp/hail.Y2Dh5hqGGSs3/6yzV4GuMR6.bm
2020-04-23 18:06:02 Hail:

Deleting pc_relate results: /tmp/tmp464njf6k/pc_relate_result
Deleting hail tmp dir: /tmp/hail.Y2Dh5hqGGSs3


2020-04-23 18:09:38 Hail: INFO: balding_nichols_model: generating genotypes for 10 populations, 6000 samples, and 6000 variants...
2020-04-23 18:09:38 Hail: INFO: Coerced sorted dataset
2020-04-23 18:09:46 Hail: INFO: wrote matrix table with 6000 rows and 6000 columns in 8 partitions to /home/rav/data/tmp/mt_6000_6000_10.mt
2020-04-23 18:09:46 Hail: INFO: hwe_normalized_pca: running PCA using 6000 variants.
2020-04-23 18:09:49 Hail: INFO: pca: running PCA with 10 components...


PCA took s:6000, v:6000, p:10 : 15.06 seconds


2020-04-23 18:10:09 Hail: INFO: Wrote all 4 blocks of 6000 x 6000 matrix with block size 4096.
2020-04-23 18:10:10 Hail: INFO: wrote matrix with 11 rows and 6000 columns as 2 blocks of size 4096 to file:/tmp/hail.Y2Dh5hqGGSs3/HyjNoUxblg.bm
2020-04-23 18:10:11 Hail: INFO: wrote matrix with 6000 rows and 6000 columns as 4 blocks of size 4096 to file:/tmp/hail.Y2Dh5hqGGSs3/egzQqUicvg.bm
2020-04-23 18:10:11 Hail: INFO: wrote matrix with 6000 rows and 6000 columns as 4 blocks of size 4096 to file:/tmp/hail.Y2Dh5hqGGSs3/wzU5K2mgLl.bm
2020-04-23 18:10:33 Hail: INFO: wrote matrix with 6000 rows and 6000 columns as 4 blocks of size 4096 to file:/tmp/hail.Y2Dh5hqGGSs3/2Xv0Efo9mz.bm
2020-04-23 18:10:53 Hail: INFO: wrote matrix with 6000 rows and 6000 columns as 4 blocks of size 4096 to file:/tmp/hail.Y2Dh5hqGGSs3/WTjrRZzauh.bm
2020-04-23 18:10:54 Hail: INFO: wrote matrix with 6000 rows and 6000 columns as 4 blocks of size 4096 to file:/tmp/hail.Y2Dh5hqGGSs3/Tm2nRZ69WW.bm
2020-04-23 18:11:19 Hail:

In [7]:
for i in range(1000, 30001, 1000):
    samples = i
    variants = samples * 5
    do_experiment("simple_1_to_5_sample_to_variant_cpu_8_ram_31", samples=samples, variants=variants, populations=10)

2020-04-23 18:28:16 Hail: INFO: hwe_normalized_pca: running PCA using 5000 variants.
2020-04-23 18:28:17 Hail: INFO: pca: running PCA with 10 components...


PCA took s:1000, v:5000, p:10 : 10.59 seconds


2020-04-23 18:28:28 Hail: INFO: Wrote all 2 blocks of 5000 x 1000 matrix with block size 4096.
2020-04-23 18:28:28 Hail: INFO: wrote matrix with 11 rows and 5000 columns as 2 blocks of size 4096 to file:/tmp/hail.1R6u1BF3Na8F/Lqs9oBTS6K.bm
2020-04-23 18:28:28 Hail: INFO: wrote matrix with 5000 rows and 1000 columns as 2 blocks of size 4096 to file:/tmp/hail.1R6u1BF3Na8F/fWzD3kRfZ9.bm
2020-04-23 18:28:29 Hail: INFO: wrote matrix with 5000 rows and 1000 columns as 2 blocks of size 4096 to file:/tmp/hail.1R6u1BF3Na8F/Mf3ETwDNJy.bm
2020-04-23 18:28:30 Hail: INFO: wrote matrix with 1000 rows and 1000 columns as 1 block of size 4096 to file:/tmp/hail.1R6u1BF3Na8F/VTdteCAnJH.bm
2020-04-23 18:28:31 Hail: INFO: wrote matrix with 1000 rows and 1000 columns as 1 block of size 4096 to file:/tmp/hail.1R6u1BF3Na8F/3adlFOsrRc.bm
2020-04-23 18:28:31 Hail: INFO: wrote matrix with 1000 rows and 1000 columns as 1 block of size 4096 to file:/tmp/hail.1R6u1BF3Na8F/AhPd8QSjUD.bm
2020-04-23 18:28:33 Hail: IN

Deleting pc_relate results: /tmp/tmpjxkgmddy/pc_relate_result
Deleting hail tmp dir: /tmp/hail.1R6u1BF3Na8F


2020-04-23 18:28:48 Hail: INFO: hwe_normalized_pca: running PCA using 10000 variants.
2020-04-23 18:28:49 Hail: INFO: pca: running PCA with 10 components...


...

2020-04-23 19:30:15 Hail: INFO: wrote table with 24496500 rows in 4 partitions to /tmp/tmpuhf2v1xm/pc_relate_result
2020-04-23 19:30:15 Hail: INFO: balding_nichols_model: generating genotypes for 10 populations, 8000 samples, and 40000 variants...
2020-04-23 19:30:16 Hail: INFO: Coerced sorted dataset
2020-04-23 19:31:02 Hail: INFO: wrote matrix table with 40000 rows and 8000 columns in 8 partitions to /home/rav/data/tmp/mt_8000_40000_10.mt
2020-04-23 19:31:06 Hail: INFO: hwe_normalized_pca: running PCA using 40000 variants.
2020-04-23 19:31:13 Hail: INFO: pca: running PCA with 10 components...


PCA took s:8000, v:40000, p:10 : 81.87 seconds


2020-04-23 19:32:49 Hail: INFO: Wrote all 20 blocks of 40000 x 8000 matrix with block size 4096.
2020-04-23 19:32:52 Hail: INFO: wrote matrix with 11 rows and 40000 columns as 10 blocks of size 4096 to file:/tmp/hail.1R6u1BF3Na8F/IJrt61HFLV.bm
2020-04-23 19:32:53 Hail: INFO: wrote matrix with 40000 rows and 8000 columns as 20 blocks of size 4096 to file:/tmp/hail.1R6u1BF3Na8F/R2dKedGNiy.bm
2020-04-23 19:33:02 Hail: INFO: wrote matrix with 40000 rows and 8000 columns as 20 blocks of size 4096 to file:/tmp/hail.1R6u1BF3Na8F/6dB2E6aKXa.bm
2020-04-23 19:35:23 Hail: INFO: wrote matrix with 8000 rows and 8000 columns as 4 blocks of size 4096 to file:/tmp/hail.1R6u1BF3Na8F/kHoBw1fPP1.bm
2020-04-23 19:37:45 Hail: INFO: wrote matrix with 8000 rows and 8000 columns as 4 blocks of size 4096 to file:/tmp/hail.1R6u1BF3Na8F/pZbGwr7sen.bm
2020-04-23 19:37:48 Hail: INFO: wrote matrix with 8000 rows and 8000 columns as 4 blocks of size 4096 to file:/tmp/hail.1R6u1BF3Na8F/5xQGdo92CQ.bm
ERROR:root:Except

Deleting hail tmp dir: /tmp/hail.1R6u1BF3Na8F


Py4JError: An error occurred while calling o806.executeJSON

In [6]:
# Fixed variants = 10k
for i in range(1000, 10001, 500):
    samples = i
    variants = 10000
    do_experiment("simple_fixed_variants_10k_cpu_8_ram_31", samples=samples, variants=variants, populations=10)

2020-04-24 02:40:26 Hail: INFO: hwe_normalized_pca: running PCA using 10000 variants.
2020-04-24 02:40:27 Hail: INFO: pca: running PCA with 10 components...


PCA took s:1000, v:10000, p:10 : 15.42 seconds


2020-04-24 02:40:40 Hail: INFO: Wrote all 3 blocks of 10000 x 1000 matrix with block size 4096.
2020-04-24 02:40:41 Hail: INFO: wrote matrix with 11 rows and 10000 columns as 3 blocks of size 4096 to file:/tmp/hail.rnK2opE09VTK/0WW9Kafui7.bm
2020-04-24 02:40:42 Hail: INFO: wrote matrix with 10000 rows and 1000 columns as 3 blocks of size 4096 to file:/tmp/hail.rnK2opE09VTK/pTFYpc5Coa.bm
2020-04-24 02:40:42 Hail: INFO: wrote matrix with 10000 rows and 1000 columns as 3 blocks of size 4096 to file:/tmp/hail.rnK2opE09VTK/YCNn0HQKqV.bm
2020-04-24 02:40:44 Hail: INFO: wrote matrix with 1000 rows and 1000 columns as 1 block of size 4096 to file:/tmp/hail.rnK2opE09VTK/1cauyIwC9W.bm
2020-04-24 02:40:46 Hail: INFO: wrote matrix with 1000 rows and 1000 columns as 1 block of size 4096 to file:/tmp/hail.rnK2opE09VTK/wGQwM3Al6W.bm
2020-04-24 02:40:46 Hail: INFO: wrote matrix with 1000 rows and 1000 columns as 1 block of size 4096 to file:/tmp/hail.rnK2opE09VTK/JER5f25v6w.bm
2020-04-24 02:40:51 Hail

Deleting pc_relate results: /tmp/tmpq31y3eut/pc_relate_result
Deleting hail tmp dir: /tmp/hail.rnK2opE09VTK


2020-04-24 02:41:09 Hail: INFO: Coerced sorted dataset
2020-04-24 02:41:12 Hail: INFO: wrote matrix table with 10000 rows and 1500 columns in 8 partitions to /home/rav/data/tmp/mt_1500_10000_10.mt
2020-04-24 02:41:12 Hail: INFO: hwe_normalized_pca: running PCA using 10000 variants.
2020-04-24 02:41:13 Hail: INFO: pca: running PCA with 10 components...


...

2020-04-24 04:02:12 Hail: INFO: balding_nichols_model: generating genotypes for 10 populations, 8500 samples, and 10000 variants...
2020-04-24 04:02:13 Hail: INFO: Coerced sorted dataset
2020-04-24 04:02:28 Hail: INFO: wrote matrix table with 10000 rows and 8500 columns in 8 partitions to /home/rav/data/tmp/mt_8500_10000_10.mt
2020-04-24 04:02:30 Hail: INFO: hwe_normalized_pca: running PCA using 10000 variants.
2020-04-24 04:02:33 Hail: INFO: pca: running PCA with 10 components...


PCA took s:8500, v:10000, p:10 : 25.76 seconds


2020-04-24 04:03:03 Hail: INFO: Wrote all 9 blocks of 10000 x 8500 matrix with block size 4096.
2020-04-24 04:03:05 Hail: INFO: wrote matrix with 11 rows and 10000 columns as 3 blocks of size 4096 to file:/tmp/hail.rnK2opE09VTK/RkOwBEGOJv.bm
2020-04-24 04:03:06 Hail: INFO: wrote matrix with 10000 rows and 8500 columns as 9 blocks of size 4096 to file:/tmp/hail.rnK2opE09VTK/MbmafMjAC9.bm
2020-04-24 04:03:08 Hail: INFO: wrote matrix with 10000 rows and 8500 columns as 9 blocks of size 4096 to file:/tmp/hail.rnK2opE09VTK/9vAjR06bgH.bm
2020-04-24 04:03:50 Hail: INFO: wrote matrix with 8500 rows and 8500 columns as 9 blocks of size 4096 to file:/tmp/hail.rnK2opE09VTK/FIxp7uKttr.bm
2020-04-24 04:04:31 Hail: INFO: wrote matrix with 8500 rows and 8500 columns as 9 blocks of size 4096 to file:/tmp/hail.rnK2opE09VTK/6qRZ26ArmM.bm
2020-04-24 04:04:32 Hail: INFO: wrote matrix with 8500 rows and 8500 columns as 9 blocks of size 4096 to file:/tmp/hail.rnK2opE09VTK/Qjcy9fZ7Z7.bm
2020-04-24 04:05:37 H

Deleting hail tmp dir: /tmp/hail.rnK2opE09VTK


Py4JError: An error occurred while calling o1451.executeJSON

In [5]:
# Fixed samples = 2k
for i in range(1000, 20001, 500):
    samples = 2000
    variants = i
    do_experiment("simple_fixed_samples_20k_cpu_8_ram_31", samples=samples, variants=variants, populations=10)

2020-04-24 05:02:30 Hail: INFO: hwe_normalized_pca: running PCA using 1000 variants.
2020-04-24 05:02:32 Hail: INFO: pca: running PCA with 10 components...


PCA took s:2000, v:1000, p:10 : 15.29 seconds


2020-04-24 05:02:45 Hail: INFO: Wrote all 1 blocks of 1000 x 2000 matrix with block size 4096.
2020-04-24 05:02:46 Hail: INFO: wrote matrix with 11 rows and 1000 columns as 1 block of size 4096 to file:/tmp/hail.sVrGxaLZ4jVW/PH75Dq8pMm.bm
2020-04-24 05:02:46 Hail: INFO: wrote matrix with 1000 rows and 2000 columns as 1 block of size 4096 to file:/tmp/hail.sVrGxaLZ4jVW/dc8a2J8YGl.bm
2020-04-24 05:02:46 Hail: INFO: wrote matrix with 1000 rows and 2000 columns as 1 block of size 4096 to file:/tmp/hail.sVrGxaLZ4jVW/D0KloM8Slr.bm
2020-04-24 05:02:47 Hail: INFO: wrote matrix with 2000 rows and 2000 columns as 1 block of size 4096 to file:/tmp/hail.sVrGxaLZ4jVW/GckgRPHU2m.bm
2020-04-24 05:02:48 Hail: INFO: wrote matrix with 2000 rows and 2000 columns as 1 block of size 4096 to file:/tmp/hail.sVrGxaLZ4jVW/ujwTW5WxU0.bm
2020-04-24 05:02:48 Hail: INFO: wrote matrix with 2000 rows and 2000 columns as 1 block of size 4096 to file:/tmp/hail.sVrGxaLZ4jVW/BtPefVYp8r.bm
2020-04-24 05:02:49 Hail: INFO:

Deleting pc_relate results: /tmp/tmp592tc38h/pc_relate_result
Deleting hail tmp dir: /tmp/hail.sVrGxaLZ4jVW


2020-04-24 05:03:31 Hail: INFO: Coerced sorted dataset
2020-04-24 05:03:33 Hail: INFO: wrote matrix table with 1500 rows and 2000 columns in 8 partitions to /home/rav/data/tmp/mt_2000_1500_10.mt
2020-04-24 05:03:33 Hail: INFO: hwe_normalized_pca: running PCA using 1500 variants.
2020-04-24 05:03:34 Hail: INFO: pca: running PCA with 10 components...


...

2020-04-24 06:13:39 Hail: INFO: Coerced sorted dataset
2020-04-24 06:13:45 Hail: INFO: wrote matrix table with 20000 rows and 2000 columns in 8 partitions to /home/rav/data/tmp/mt_2000_20000_10.mt
2020-04-24 06:13:46 Hail: INFO: hwe_normalized_pca: running PCA using 20000 variants.
2020-04-24 06:13:47 Hail: INFO: pca: running PCA with 10 components...


PCA took s:2000, v:20000, p:10 : 17.53 seconds


2020-04-24 06:14:06 Hail: INFO: Wrote all 5 blocks of 20000 x 2000 matrix with block size 4096.
2020-04-24 06:14:07 Hail: INFO: wrote matrix with 11 rows and 20000 columns as 5 blocks of size 4096 to file:/tmp/hail.sVrGxaLZ4jVW/xvRDav5RKk.bm
2020-04-24 06:14:07 Hail: INFO: wrote matrix with 20000 rows and 2000 columns as 5 blocks of size 4096 to file:/tmp/hail.sVrGxaLZ4jVW/MAhBor4i0r.bm
2020-04-24 06:14:08 Hail: INFO: wrote matrix with 20000 rows and 2000 columns as 5 blocks of size 4096 to file:/tmp/hail.sVrGxaLZ4jVW/Cm7X9vVciy.bm
2020-04-24 06:14:25 Hail: INFO: wrote matrix with 2000 rows and 2000 columns as 1 block of size 4096 to file:/tmp/hail.sVrGxaLZ4jVW/pvc2EI5iZm.bm
2020-04-24 06:14:42 Hail: INFO: wrote matrix with 2000 rows and 2000 columns as 1 block of size 4096 to file:/tmp/hail.sVrGxaLZ4jVW/lwWt7sP8SD.bm
2020-04-24 06:14:42 Hail: INFO: wrote matrix with 2000 rows and 2000 columns as 1 block of size 4096 to file:/tmp/hail.sVrGxaLZ4jVW/3KFczzRezT.bm
2020-04-24 06:15:04 Hail

Deleting pc_relate results: /tmp/tmp0h0cigug/pc_relate_result
Deleting hail tmp dir: /tmp/hail.sVrGxaLZ4jVW


2020-04-24 06:16:35 Hail: INFO: wrote table with 1999000 rows in 1 partition to /tmp/tmp0h0cigug/pc_relate_result


In [ ]:
# Fixed variants = 2k
for i in range(1000, 20001, 500):
    samples = i
    variants = 2000
    do_experiment("simple_fixed_variants_2k_cpu_8_ram_31", samples=samples, variants=variants, populations=10)

2020-04-24 14:57:19 Hail: INFO: hwe_normalized_pca: running PCA using 2000 variants.
2020-04-24 14:57:19 Hail: INFO: pca: running PCA with 10 components...


PCA took s:1000, v:2000, p:10 : 7.21 seconds


2020-04-24 14:57:27 Hail: INFO: Wrote all 1 blocks of 2000 x 1000 matrix with block size 4096.
2020-04-24 14:57:27 Hail: INFO: wrote matrix with 11 rows and 2000 columns as 1 block of size 4096 to file:/tmp/hail.UCzZIpmxAOC7/eVLXPOQ6qW.bm
2020-04-24 14:57:27 Hail: INFO: wrote matrix with 2000 rows and 1000 columns as 1 block of size 4096 to file:/tmp/hail.UCzZIpmxAOC7/eUFGAPx9Xo.bm
2020-04-24 14:57:28 Hail: INFO: wrote matrix with 2000 rows and 1000 columns as 1 block of size 4096 to file:/tmp/hail.UCzZIpmxAOC7/BthR51qQdV.bm
2020-04-24 14:57:28 Hail: INFO: wrote matrix with 1000 rows and 1000 columns as 1 block of size 4096 to file:/tmp/hail.UCzZIpmxAOC7/1BduGmDDwo.bm
2020-04-24 14:57:29 Hail: INFO: wrote matrix with 1000 rows and 1000 columns as 1 block of size 4096 to file:/tmp/hail.UCzZIpmxAOC7/kSzyzvMRHo.bm
2020-04-24 14:57:29 Hail: INFO: wrote matrix with 1000 rows and 1000 columns as 1 block of size 4096 to file:/tmp/hail.UCzZIpmxAOC7/Y3WmOJJn4d.bm
2020-04-24 14:57:29 Hail: INFO:

Deleting pc_relate results: /tmp/tmp1uuw5caf/pc_relate_result


2020-04-24 14:57:42 Hail: INFO: wrote matrix with 11 rows and 2000 columns as 1 block of size 4096 to file:/tmp/hail.UCzZIpmxAOC7/BgW6VYjLOW.bm
2020-04-24 14:57:42 Hail: INFO: wrote matrix with 2000 rows and 1000 columns as 1 block of size 4096 to file:/tmp/hail.UCzZIpmxAOC7/JQhr7hasEP.bm
2020-04-24 14:57:42 Hail: INFO: wrote matrix with 2000 rows and 1000 columns as 1 block of size 4096 to file:/tmp/hail.UCzZIpmxAOC7/nJlZ2UwwDo.bm
2020-04-24 14:57:42 Hail: INFO: wrote matrix with 1000 rows and 1000 columns as 1 block of size 4096 to file:/tmp/hail.UCzZIpmxAOC7/C1lpgx6a3n.bm
2020-04-24 14:57:43 Hail: INFO: wrote matrix with 1000 rows and 1000 columns as 1 block of size 4096 to file:/tmp/hail.UCzZIpmxAOC7/8vvJDNiOoH.bm
2020-04-24 14:57:43 Hail: INFO: wrote matrix with 1000 rows and 1000 columns as 1 block of size 4096 to file:/tmp/hail.UCzZIpmxAOC7/Sqo9sI9Kea.bm
2020-04-24 14:57:44 Hail: INFO: wrote matrix with 1000 rows and 1000 columns as 1 block of size 4096 to file:/tmp/hail.UCzZIpm

...

2020-04-24 15:47:29 Hail: INFO: wrote matrix with 11 rows and 2000 columns as 1 block of size 4096 to file:/tmp/hail.UCzZIpmxAOC7/VGymu8GdGj.bm
2020-04-24 15:47:29 Hail: INFO: wrote matrix with 2000 rows and 7000 columns as 2 blocks of size 4096 to file:/tmp/hail.UCzZIpmxAOC7/F6oFP8askd.bm
2020-04-24 15:47:30 Hail: INFO: wrote matrix with 2000 rows and 7000 columns as 2 blocks of size 4096 to file:/tmp/hail.UCzZIpmxAOC7/DhtP8PbsEY.bm
2020-04-24 15:47:37 Hail: INFO: wrote matrix with 7000 rows and 7000 columns as 4 blocks of size 4096 to file:/tmp/hail.UCzZIpmxAOC7/7d2lOCTy8L.bm
2020-04-24 15:47:45 Hail: INFO: wrote matrix with 7000 rows and 7000 columns as 4 blocks of size 4096 to file:/tmp/hail.UCzZIpmxAOC7/Bt3J4JMQU4.bm
2020-04-24 15:47:45 Hail: INFO: wrote matrix with 7000 rows and 7000 columns as 4 blocks of size 4096 to file:/tmp/hail.UCzZIpmxAOC7/gxgLB8sdAJ.bm
2020-04-24 15:47:54 Hail: INFO: wrote matrix with 7000 rows and 7000 columns as 4 blocks of size 4096 to file:/tmp/hail.U

Deleting hail tmp dir: /tmp/hail.UCzZIpmxAOC7


2020-04-24 15:50:40 Hail: INFO: hwe_normalized_pca: running PCA using 2000 variants.
2020-04-24 15:50:42 Hail: INFO: pca: running PCA with 10 components...


PCA took s:7500, v:2000, p:10 : 13.52 seconds


2020-04-24 15:50:58 Hail: INFO: Wrote all 2 blocks of 2000 x 7500 matrix with block size 4096.
2020-04-24 15:51:00 Hail: INFO: wrote matrix with 11 rows and 2000 columns as 1 block of size 4096 to file:/tmp/hail.UCzZIpmxAOC7/Sxm2MsK8SG.bm
2020-04-24 15:51:00 Hail: INFO: wrote matrix with 2000 rows and 7500 columns as 2 blocks of size 4096 to file:/tmp/hail.UCzZIpmxAOC7/I6K64JaHQI.bm
2020-04-24 15:51:00 Hail: INFO: wrote matrix with 2000 rows and 7500 columns as 2 blocks of size 4096 to file:/tmp/hail.UCzZIpmxAOC7/cjBFJOMsRN.bm
2020-04-24 15:51:08 Hail: INFO: wrote matrix with 7500 rows and 7500 columns as 4 blocks of size 4096 to file:/tmp/hail.UCzZIpmxAOC7/J7RsdoQHXs.bm
2020-04-24 15:51:15 Hail: INFO: wrote matrix with 7500 rows and 7500 columns as 4 blocks of size 4096 to file:/tmp/hail.UCzZIpmxAOC7/EsKLm2iAe9.bm
2020-04-24 15:51:16 Hail: INFO: wrote matrix with 7500 rows and 7500 columns as 4 blocks of size 4096 to file:/tmp/hail.UCzZIpmxAOC7/nP1XXClwfS.bm
2020-04-24 15:51:25 Hail: 

In [ ]:
# Fixed variants = 2k
for i in range(1000, 20001, 500):
    samples = i
    variants = 2000
    do_experiment("simple_fixed_variants_2k_cpu_8_ram_31_blocksize_2048", samples=samples, variants=variants, populations=10)

In [ ]:
# Fixed variants = 2k
for i in range(1000, 20001, 200):
    samples = i
    variants = 2000
    do_experiment("simple_fixed_variants_2k_cpu_8_ram_31_blocksize_512", samples=samples, variants=variants, populations=10)

2020-04-24 16:44:16 Hail: INFO: hwe_normalized_pca: running PCA using 2000 variants.
2020-04-24 16:44:18 Hail: INFO: pca: running PCA with 10 components...


PCA took s:1000, v:2000, p:10 : 13.00 seconds


2020-04-24 16:44:28 Hail: INFO: Wrote all 8 blocks of 2000 x 1000 matrix with block size 512.
2020-04-24 16:44:29 Hail: INFO: wrote matrix with 11 rows and 2000 columns as 4 blocks of size 512 to file:/tmp/hail.lSI8hItnvyNe/33ARiL6xj9.bm
2020-04-24 16:44:29 Hail: INFO: wrote matrix with 2000 rows and 1000 columns as 8 blocks of size 512 to file:/tmp/hail.lSI8hItnvyNe/slk16q8ucA.bm
2020-04-24 16:44:30 Hail: INFO: wrote matrix with 2000 rows and 1000 columns as 8 blocks of size 512 to file:/tmp/hail.lSI8hItnvyNe/xLyapBK4O4.bm
2020-04-24 16:44:30 Hail: INFO: wrote matrix with 1000 rows and 1000 columns as 4 blocks of size 512 to file:/tmp/hail.lSI8hItnvyNe/AjLLx3cWJ7.bm
2020-04-24 16:44:30 Hail: INFO: wrote matrix with 1000 rows and 1000 columns as 4 blocks of size 512 to file:/tmp/hail.lSI8hItnvyNe/eQ3PT2kslR.bm
2020-04-24 16:44:31 Hail: INFO: wrote matrix with 1000 rows and 1000 columns as 4 blocks of size 512 to file:/tmp/hail.lSI8hItnvyNe/nkfCeLw0TU.bm
2020-04-24 16:44:31 Hail: INFO: 

Deleting pc_relate results: /tmp/tmpxokdn9ut/pc_relate_result
Deleting hail tmp dir: /tmp/hail.lSI8hItnvyNe


2020-04-24 16:44:38 Hail: INFO: Coerced sorted dataset
2020-04-24 16:44:39 Hail: INFO: wrote matrix table with 2000 rows and 1200 columns in 8 partitions to /home/rav/data/tmp/mt_1200_2000_10.mt
2020-04-24 16:44:39 Hail: INFO: hwe_normalized_pca: running PCA using 2000 variants.
2020-04-24 16:44:40 Hail: INFO: pca: running PCA with 10 components...


...

2020-04-24 17:07:07 Hail: INFO: Wrote all 48 blocks of 2000 x 6000 matrix with block size 512.
2020-04-24 17:07:08 Hail: INFO: wrote matrix with 11 rows and 2000 columns as 4 blocks of size 512 to file:/tmp/hail.lSI8hItnvyNe/YnbcgISNCp.bm
2020-04-24 17:07:09 Hail: INFO: wrote matrix with 2000 rows and 6000 columns as 48 blocks of size 512 to file:/tmp/hail.lSI8hItnvyNe/HqeZfEwJjR.bm
2020-04-24 17:07:09 Hail: INFO: wrote matrix with 2000 rows and 6000 columns as 48 blocks of size 512 to file:/tmp/hail.lSI8hItnvyNe/TTLWmmAC6c.bm
2020-04-24 17:07:13 Hail: INFO: wrote matrix with 6000 rows and 6000 columns as 144 blocks of size 512 to file:/tmp/hail.lSI8hItnvyNe/fVygPL1NJq.bm
2020-04-24 17:07:18 Hail: INFO: wrote matrix with 6000 rows and 6000 columns as 144 blocks of size 512 to file:/tmp/hail.lSI8hItnvyNe/2dBSZLKfzi.bm
2020-04-24 17:07:18 Hail: INFO: wrote matrix with 6000 rows and 6000 columns as 144 blocks of size 512 to file:/tmp/hail.lSI8hItnvyNe/BKDc1lczsy.bm
2020-04-24 17:07:23 Hai

Deleting pc_relate results: /tmp/tmpobmumy_t/pc_relate_result
Deleting hail tmp dir: /tmp/hail.lSI8hItnvyNe


2020-04-24 17:08:51 Hail: INFO: balding_nichols_model: generating genotypes for 10 populations, 6200 samples, and 2000 variants...
2020-04-24 17:08:52 Hail: INFO: Coerced sorted dataset
2020-04-24 17:08:55 Hail: INFO: wrote matrix table with 2000 rows and 6200 columns in 8 partitions to /home/rav/data/tmp/mt_6200_2000_10.mt
2020-04-24 17:08:55 Hail: INFO: hwe_normalized_pca: running PCA using 2000 variants.
2020-04-24 17:08:56 Hail: INFO: pca: running PCA with 10 components...


PCA took s:6200, v:2000, p:10 : 10.02 seconds


2020-04-24 17:09:08 Hail: INFO: Wrote all 52 blocks of 2000 x 6200 matrix with block size 512.
2020-04-24 17:09:09 Hail: INFO: wrote matrix with 11 rows and 2000 columns as 4 blocks of size 512 to file:/tmp/hail.lSI8hItnvyNe/E9o2P3hrZM.bm
2020-04-24 17:09:09 Hail: INFO: wrote matrix with 2000 rows and 6200 columns as 52 blocks of size 512 to file:/tmp/hail.lSI8hItnvyNe/nmsSWSzSX0.bm
2020-04-24 17:09:10 Hail: INFO: wrote matrix with 2000 rows and 6200 columns as 52 blocks of size 512 to file:/tmp/hail.lSI8hItnvyNe/aKLB7rkkun.bm
2020-04-24 17:09:14 Hail: INFO: wrote matrix with 6200 rows and 6200 columns as 169 blocks of size 512 to file:/tmp/hail.lSI8hItnvyNe/MrmCmpaxEi.bm


In [ ]:
# Fixed variants = 2k
for i in range(1000, 20001, 200):
    samples = i
    variants = 2000
    do_experiment("simple_fixed_variants_2k_cpu_8_ram_31_blocksize_eq_sample", samples=samples, variants=variants, populations=10, block_size=samples)
    

2020-04-24 17:18:43 Hail: INFO: hwe_normalized_pca: running PCA using 2000 variants.
2020-04-24 17:18:43 Hail: INFO: pca: running PCA with 10 components...


PCA took s:1000, v:2000, p:10 : 8.29 seconds


2020-04-24 17:18:51 Hail: INFO: Wrote all 2 blocks of 2000 x 1000 matrix with block size 1000.
2020-04-24 17:18:52 Hail: INFO: wrote matrix with 11 rows and 2000 columns as 2 blocks of size 1000 to file:/tmp/hail.GXgR4Pd2q9HV/uvqvcTgvDW.bm
2020-04-24 17:18:52 Hail: INFO: wrote matrix with 2000 rows and 1000 columns as 2 blocks of size 1000 to file:/tmp/hail.GXgR4Pd2q9HV/nViEZqK1nM.bm
2020-04-24 17:18:52 Hail: INFO: wrote matrix with 2000 rows and 1000 columns as 2 blocks of size 1000 to file:/tmp/hail.GXgR4Pd2q9HV/dGaco7Fqx7.bm
2020-04-24 17:18:53 Hail: INFO: wrote matrix with 1000 rows and 1000 columns as 1 block of size 1000 to file:/tmp/hail.GXgR4Pd2q9HV/vVeb9oLnkB.bm
2020-04-24 17:18:54 Hail: INFO: wrote matrix with 1000 rows and 1000 columns as 1 block of size 1000 to file:/tmp/hail.GXgR4Pd2q9HV/JqSrYd9hXW.bm
2020-04-24 17:18:54 Hail: INFO: wrote matrix with 1000 rows and 1000 columns as 1 block of size 1000 to file:/tmp/hail.GXgR4Pd2q9HV/QvuucBDBH1.bm
2020-04-24 17:18:54 Hail: IN

Deleting pc_relate results: /tmp/tmpu1_6jn7c/pc_relate_result
Deleting hail tmp dir: /tmp/hail.GXgR4Pd2q9HV


2020-04-24 17:19:06 Hail: INFO: hwe_normalized_pca: running PCA using 2000 variants.
2020-04-24 17:19:07 Hail: INFO: pca: running PCA with 10 components...


...

2020-04-24 17:43:47 Hail: INFO: Wrote all 1 blocks of 2000 x 4200 matrix with block size 4200.
2020-04-24 17:43:48 Hail: INFO: wrote matrix with 11 rows and 2000 columns as 1 block of size 4200 to file:/tmp/hail.GXgR4Pd2q9HV/XwDSd0PzC8.bm
2020-04-24 17:43:48 Hail: INFO: wrote matrix with 2000 rows and 4200 columns as 1 block of size 4200 to file:/tmp/hail.GXgR4Pd2q9HV/K3e5WUfrTU.bm
2020-04-24 17:43:49 Hail: INFO: wrote matrix with 2000 rows and 4200 columns as 1 block of size 4200 to file:/tmp/hail.GXgR4Pd2q9HV/Fkg3xaWZKX.bm
2020-04-24 17:43:56 Hail: INFO: wrote matrix with 4200 rows and 4200 columns as 1 block of size 4200 to file:/tmp/hail.GXgR4Pd2q9HV/v8KU2ffT65.bm
2020-04-24 17:44:04 Hail: INFO: wrote matrix with 4200 rows and 4200 columns as 1 block of size 4200 to file:/tmp/hail.GXgR4Pd2q9HV/VTHiC9kS7y.bm
2020-04-24 17:44:04 Hail: INFO: wrote matrix with 4200 rows and 4200 columns as 1 block of size 4200 to file:/tmp/hail.GXgR4Pd2q9HV/d4IVUQ1jbv.bm
2020-04-24 17:44:13 Hail: INFO:

Deleting pc_relate results: /tmp/tmp6t0a3dvl/pc_relate_result
Deleting hail tmp dir: /tmp/hail.GXgR4Pd2q9HV


2020-04-24 17:47:21 Hail: INFO: hwe_normalized_pca: running PCA using 2000 variants.
2020-04-24 17:47:22 Hail: INFO: pca: running PCA with 10 components...


PCA took s:4400, v:2000, p:10 : 9.19 seconds


2020-04-24 17:47:32 Hail: INFO: Wrote all 1 blocks of 2000 x 4400 matrix with block size 4400.
2020-04-24 17:47:33 Hail: INFO: wrote matrix with 11 rows and 2000 columns as 1 block of size 4400 to file:/tmp/hail.GXgR4Pd2q9HV/3KLMtloCQp.bm
2020-04-24 17:47:34 Hail: INFO: wrote matrix with 2000 rows and 4400 columns as 1 block of size 4400 to file:/tmp/hail.GXgR4Pd2q9HV/S8tMyjgLmD.bm
2020-04-24 17:47:34 Hail: INFO: wrote matrix with 2000 rows and 4400 columns as 1 block of size 4400 to file:/tmp/hail.GXgR4Pd2q9HV/wcehHzGaAZ.bm
2020-04-24 17:47:42 Hail: INFO: wrote matrix with 4400 rows and 4400 columns as 1 block of size 4400 to file:/tmp/hail.GXgR4Pd2q9HV/5mYOKLjvNE.bm


In [ ]:
# Fixed variants = 2k
for i in range(1000, 20001, 200):
    samples = i
    variants = 2000
    # 8 cores
    block_size = int(samples/8) + 1
    do_experiment("simple_fixed_variants_2k_cpu_8_ram_31_blocksize_per_cpu", samples=samples, variants=variants, populations=10, block_size=block_size)

2020-04-24 18:05:24 Hail: INFO: hwe_normalized_pca: running PCA using 2000 variants.
2020-04-24 18:05:25 Hail: INFO: pca: running PCA with 10 components...


PCA took s:1000, v:2000, p:10 : 13.09 seconds


2020-04-24 18:05:36 Hail: INFO: Wrote all 128 blocks of 2000 x 1000 matrix with block size 126.
2020-04-24 18:05:37 Hail: INFO: wrote matrix with 11 rows and 2000 columns as 16 blocks of size 126 to file:/tmp/hail.pcWkcZDbNULq/xinM3fOuiN.bm
2020-04-24 18:05:38 Hail: INFO: wrote matrix with 2000 rows and 1000 columns as 128 blocks of size 126 to file:/tmp/hail.pcWkcZDbNULq/UeZVulIUDQ.bm
2020-04-24 18:05:38 Hail: INFO: wrote matrix with 2000 rows and 1000 columns as 128 blocks of size 126 to file:/tmp/hail.pcWkcZDbNULq/OFsUIGCCXm.bm
2020-04-24 18:05:39 Hail: INFO: wrote matrix with 1000 rows and 1000 columns as 64 blocks of size 126 to file:/tmp/hail.pcWkcZDbNULq/SbSArhyVEy.bm
2020-04-24 18:05:39 Hail: INFO: wrote matrix with 1000 rows and 1000 columns as 64 blocks of size 126 to file:/tmp/hail.pcWkcZDbNULq/dlJfJfQfXk.bm
2020-04-24 18:05:40 Hail: INFO: wrote matrix with 1000 rows and 1000 columns as 64 blocks of size 126 to file:/tmp/hail.pcWkcZDbNULq/zQIEO4zWjF.bm
2020-04-24 18:05:40 Ha

Deleting pc_relate results: /tmp/tmplwhv7trh/pc_relate_result
Deleting hail tmp dir: /tmp/hail.pcWkcZDbNULq
Deleting hail tmp dir: /tmp/hail.OfFY53OXkvYK
Deleting hail tmp dir: /tmp/hail.G8Mqn1St2G43


2020-04-24 18:05:49 Hail: INFO: hwe_normalized_pca: running PCA using 2000 variants.
2020-04-24 18:05:50 Hail: INFO: pca: running PCA with 10 components...


...

2020-04-24 19:49:57 Hail: INFO: Wrote all 16 blocks of 2000 x 10600 matrix with block size 1326.
2020-04-24 19:49:59 Hail: INFO: wrote matrix with 11 rows and 2000 columns as 2 blocks of size 1326 to file:/tmp/hail.pcWkcZDbNULq/TVkMTwDjXW.bm
2020-04-24 19:50:00 Hail: INFO: wrote matrix with 2000 rows and 10600 columns as 16 blocks of size 1326 to file:/tmp/hail.pcWkcZDbNULq/A6dR1apDqo.bm
2020-04-24 19:50:00 Hail: INFO: wrote matrix with 2000 rows and 10600 columns as 16 blocks of size 1326 to file:/tmp/hail.pcWkcZDbNULq/MJajaXFAhI.bm
2020-04-24 19:50:13 Hail: INFO: wrote matrix with 10600 rows and 10600 columns as 64 blocks of size 1326 to file:/tmp/hail.pcWkcZDbNULq/7H0rFWo0kn.bm
2020-04-24 19:50:25 Hail: INFO: wrote matrix with 10600 rows and 10600 columns as 64 blocks of size 1326 to file:/tmp/hail.pcWkcZDbNULq/tJJEZm63bP.bm
2020-04-24 19:50:26 Hail: INFO: wrote matrix with 10600 rows and 10600 columns as 64 blocks of size 1326 to file:/tmp/hail.pcWkcZDbNULq/sr4msbeewH.bm
2020-04-24

Deleting pc_relate results: /tmp/tmpqx_xypfr/pc_relate_result
Deleting hail tmp dir: /tmp/hail.pcWkcZDbNULq


2020-04-24 19:55:32 Hail: INFO: balding_nichols_model: generating genotypes for 10 populations, 10800 samples, and 2000 variants...
2020-04-24 19:55:33 Hail: INFO: Coerced sorted dataset
2020-04-24 19:55:38 Hail: INFO: wrote matrix table with 2000 rows and 10800 columns in 8 partitions to /home/rav/data/tmp/mt_10800_2000_10.mt
2020-04-24 19:55:39 Hail: INFO: hwe_normalized_pca: running PCA using 2000 variants.
2020-04-24 19:55:41 Hail: INFO: pca: running PCA with 10 components...


PCA took s:10800, v:2000, p:10 : 13.56 seconds


2020-04-24 19:55:57 Hail: INFO: Wrote all 16 blocks of 2000 x 10800 matrix with block size 1351.
2020-04-24 19:55:59 Hail: INFO: wrote matrix with 11 rows and 2000 columns as 2 blocks of size 1351 to file:/tmp/hail.pcWkcZDbNULq/hmz2s89BWW.bm
2020-04-24 19:55:59 Hail: INFO: wrote matrix with 2000 rows and 10800 columns as 16 blocks of size 1351 to file:/tmp/hail.pcWkcZDbNULq/hSalKap851.bm
2020-04-24 19:56:00 Hail: INFO: wrote matrix with 2000 rows and 10800 columns as 16 blocks of size 1351 to file:/tmp/hail.pcWkcZDbNULq/w7mCGeQqTp.bm
2020-04-24 19:56:13 Hail: INFO: wrote matrix with 10800 rows and 10800 columns as 64 blocks of size 1351 to file:/tmp/hail.pcWkcZDbNULq/AZqWcq1tKU.bm
2020-04-24 19:56:26 Hail: INFO: wrote matrix with 10800 rows and 10800 columns as 64 blocks of size 1351 to file:/tmp/hail.pcWkcZDbNULq/5ZWvNRiV3O.bm
2020-04-24 19:56:27 Hail: INFO: wrote matrix with 10800 rows and 10800 columns as 64 blocks of size 1351 to file:/tmp/hail.pcWkcZDbNULq/xoOqMQA7zx.bm
2020-04-24

In [ ]:
# pure spark pca
from pyspark.mllib.linalg.distributed import RowMatrix
#from pyspark.mllib.random.RandomRDDs import RandomRDDs
from pyspark.mllib.random import RandomRDDs

sc = hl.spark_context()
mat = RowMatrix(RandomRDDs.uniformVectorRDD(sc, 5000, 5000, numPartitions=200))
pc = mat.computePrincipalComponents(10)
projected = mat.multiply(pc)